In [1]:
device = "mps"

In [2]:
from models.attention_exp_LSTM.dataset import energyProject_dataset
from models.attention_exp_LSTM.network import attentiveLSTM_model

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import pandas as pd
import yaml
import wandb
from typing import Dict
import matplotlib.pyplot as plt

In [3]:
def nested_dict(original_dict):
    nested_dict = {}
    for key, value in original_dict.items():
        parts = key.split(".")
        d = nested_dict
        for part in parts[:-1]:
            if part not in d:
                d[part] = {}
            d = d[part]
        d[parts[-1]] = value
    return nested_dict


#load dataset object file
with (open('data/dataset_class.pkl', "rb")) as openfile:
    dataset = pickle.load(openfile)

dataset.activitivity_encoding_mode = 2 #or any value
# Splitting the dataset into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42, shuffle=False )

train_dataloader = DataLoader(train_data, batch_size=100, shuffle=False) #!!! useful to shuffle?
val_dataloader = DataLoader(val_data, batch_size=100, shuffle=False)
whole_dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

In [4]:
#setup wandb stuff
with open('models/attention_exp_LSTM/config.yaml', 'r') as stream:
    try:
        sweep_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Sweep to find best hyperparams:

In [5]:
y_idx=0

In [6]:
scaler = dataset.column_scalers[dataset.target_labels[y_idx]]

In [9]:
def train(config: Dict = None):
    with wandb.init(config=config) as run:
        run.name = "ENERGY_METER_0"
        config = wandb.config
        config = nested_dict(config)
        optimizer_config = config["optimizer"]
        model = attentiveLSTM_model(espai_emb_dim=config["espai_emb_dim"],hidden_dim=config['hidden_size'],lstm_nl=config['num_layers'],nheads=config['heads'],attnFCdim=config['attnFCdim'])
        model.init_weights()
        model.to(device)

        criterion =  nn.MSELoss()

        if optimizer_config["type"] == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr = optimizer_config['lr'])

        num_epochs = 80
        best_loss = float('inf')
        for epoch in range(num_epochs):
            model.train()
            training_losses = [] # renamed from epoch_losses
            progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}")
            for batch,data in progress_bar:
                ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
                y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

                optimizer.zero_grad()
                
                #current batch size size
                b_sz = ocu_emb.shape[0]

                #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
                h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
                h = h.to(device)
                c = c.to(device)
                y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
                y_pred = y_pred[:,0,0]

                loss = criterion(y_pred,y)  #cross entropy loss needs (N,C,seq_lenght)
                loss.backward()
                optimizer.step()    

                training_losses.append(loss.item())
                progress_bar.set_postfix({'Batch Loss': loss.item()})

            average_training_loss = sum(training_losses) / len(training_losses) # renamed from avg_loss
            #average_training_loss = np.power(dataset.denormalize_values(np.sqrt(average_training_loss),scaler),2)
            wandb.log({'Train_Epoch_Loss': average_training_loss})

            model.eval()  
            with torch.no_grad():  
                validation_losses = [] # renamed from val_losses
                for batch in tqdm(val_dataloader, desc='Validation'):
                    ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
                    y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

                    #current batch size size
                    b_sz = ocu_emb.shape[0]

                    #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
                    h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
                    h = h.to(device)
                    c = c.to(device)
                    y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
                    y_pred = y_pred[:,0,0]
                    
                    loss = criterion(y_pred,y) 
                    validation_losses.append(loss.item())

                average_validation_loss = sum(validation_losses) / len(validation_losses) # renamed from avg_val_loss
                #average_validation_loss = np.power(dataset.denormalize_values(np.sqrt(average_validation_loss),scaler),2)
                wandb.log({'Validation_Epoch_Loss': average_validation_loss})

            if average_training_loss < best_loss:
                best_loss = average_training_loss
                torch.save(model.state_dict(), 'models/attention_exp_LSTM/tranformLSTM.pt')
                wandb.save('gru_model.pt')
                print(f"Model saved at {'gru_model.pt'}")

        wandb.finish()

In [10]:
!wandb login

wandb: Currently logged in as: haradai (energy_project_uab). Use `wandb login --relogin` to force relogin


In [11]:
sweep_id = wandb.sweep(sweep_config, project="energy_project_uab")
sweep_id

Create sweep with ID: aic13z0d
Sweep URL: https://wandb.ai/energy_project_uab/energy_project_uab/sweeps/aic13z0d


'aic13z0d'

In [12]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 2c7x004j with config:
wandb: 	attnFCdim: 1000
wandb: 	espai_emb_dim: 100
wandb: 	heads: 10
wandb: 	hidden_size: 1000
wandb: 	num_layers: 1
wandb: 	optimizer: {'lr': 0.00711967013233076, 'type': 'adam'}
wandb: Currently logged in as: haradai (energy_project_uab). Use `wandb login --relogin` to force relogin


Epoch 1/80: 71it [01:04,  1.09it/s, Batch Loss=0.102] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 23.92it/s]


Model saved at gru_model.pt


Epoch 2/80: 71it [01:05,  1.08it/s, Batch Loss=0.174] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 22.74it/s]


Model saved at gru_model.pt


...

Epoch 20/80: 71it [01:03,  1.12it/s, Batch Loss=0.0125]
Validation: 100%|██████████| 18/18 [00:00<00:00, 24.89it/s]


Model saved at gru_model.pt


Epoch 21/80: 71it [01:03,  1.12it/s, Batch Loss=0.0108]
Validation: 100%|██████████| 18/18 [00:00<00:00, 23.38it/s]


Model saved at gru_model.pt


Epoch 22/80: 71it [01:02,  1.13it/s, Batch Loss=0.0181]
Validation: 100%|██████████| 18/18 [00:00<00:00, 24.54it/s]
Epoch 23/80: 71it [01:02,  1.13it/s, Batch Loss=0.0163]
Validation: 100%|██████████| 18/18 [00:00<00:00, 24.82it/s]
Epoch 24/80: 71it [01:02,  1.13it/s, Batch Loss=0.0206]
Validation: 100%|██████████| 18/18 [00:00<00:00, 23.59it/s]
Epoch 25/80: 71it [01:03,  1.12it/s, Batch Loss=0.0154]
Validation: 100%|██████████| 18/18 [00:00<00:00, 25.30it/s]
Epoch 26/80: 71it [01:03,  1.12it/s, Batch Loss=0.0173]
Validation: 100%|██████████| 18/18 [00:00<00:00, 24.04it/s]
Epoch 27/80: 71it [01:03,  1.11it/s, Batch Loss=0.0135]
Validation: 100%|██████████| 18/18 [00:00<00:00, 23.94it/s]
Epoch 28/80: 71it [01:03,  1.12it/s, Batch Loss=0.0116]
Validation: 100%|██████████| 18/18 [00:00<00:00, 23.25it/s]
Epoch 29/80: 71it [01:03,  1.12it/s, Batch Loss=0.0118]
Validation: 100%|██████████| 18/18 [00:00<00:00, 23.77it/s]
Epoch 30/80: 71it [01:04,  1.10it/s, Batch Loss=0.00478]
Validation: 100

In [4]:
def predict_consumption(model,dataloader,y_idx):
    real = []
    pred = []
    h, c = model.init_hidden(dataloader.batch_size) # Start with a new state in each batch            
    h = h.to(device)
    c = c.to(device)
    for batch, data in enumerate(dataloader):
        with torch.no_grad():
            #current batch size size
            ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
            y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now
            
            b_sz = ocu_emb.shape[0]
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)

            y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            pred += list(y_pred[:,0,0].to("cpu"))
            real += list(y.to("cpu"))
    return real ,pred

From out sweep these are the best parameters found: Let's train again such a model but for many more epochs:

Note that we made a change in the model code as we added 0.2 dropout probability to the lstm, this is due to that lstm tends to overfit. We hope that with this change the model will generalize better the data.

We'll train one model for each energy counter:

# y = 0

In [ ]:
y_idx = 0
model = attentiveLSTM_model(espai_emb_dim=50,hidden_dim=384,lstm_nl=1,nheads=2,attnFCdim=80)
model.init_weights()
model.to(device)

criterion =  nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #lr used in that 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=(1 - 0.0001 / 0.001))
num_epochs = 200
best_loss = float('inf')
epoch_loss_tr = []
epoch_loss_vl = []
with wandb.init(project="energy_project_uab", entity = "energy_project_uab") as run:
    run.name = f"LSTM_attention_regressor_0"
    for epoch in range(num_epochs):
        model.train()
        training_losses = [] # renamed from epoch_losses
        progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}")
        for batch,data in progress_bar:
            ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
            y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

            optimizer.zero_grad()
            
            #current batch size size
            b_sz = ocu_emb.shape[0]

            #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)
            y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            y_pred = y_pred[:,0,0]

            loss = criterion(y_pred,y)  #cross entropy loss needs (N,C,seq_lenght)
            loss.backward()
            optimizer.step()    

            training_losses.append(loss.item())
            progress_bar.set_postfix({'Batch Loss': loss.item()})

        average_training_loss = sum(training_losses) / len(training_losses) # renamed from avg_loss
        epoch_loss_tr.append(average_training_loss)
        wandb.log({"Train_loss":average_training_loss})
        #average_training_loss = np.power(dataset.denormalize_values(np.sqrt(average_training_loss),scaler),2)

        model.eval()  
        with torch.no_grad():  
            validation_losses = [] # renamed from val_losses
            for batch in tqdm(val_dataloader, desc='Validation'):
                ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
                y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

                #current batch size size
                b_sz = ocu_emb.shape[0]

                #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
                h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
                h = h.to(device)
                c = c.to(device)
                y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
                y_pred = y_pred[:,0,0]
                
                loss = criterion(y_pred,y) 
                validation_losses.append(loss.item())

            average_validation_loss = sum(validation_losses) / len(validation_losses) # renamed from avg_val_loss
            #average_validation_loss = np.power(dataset.denormalize_values(np.sqrt(average_validation_loss),scaler),2)
            epoch_loss_vl.append(average_validation_loss)
            wandb.log({"Val_loss":average_validation_loss})
        
        scheduler.step() #change lr
        print(f"lr:{scheduler.get_last_lr()}")#print current lr

        if average_validation_loss < best_loss:
            best_loss = average_training_loss
            torch.save(model.state_dict(), 'models/attention_exp_LSTM/LSTM_attention_regressor_0.pt')
            wandb.save('models/attention_exp_LSTM/LSTM_attention_regressor_0.pt')
            print("Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_0.pt")
        
        if epoch == num_epochs-1: #if last epoch just finished
            print("Running whole dataset prediction")
            real, pred = predict_consumption(model,whole_dataloader,y_idx=0)
            real = [v.item() for v in real]
            pred = [v.item() for v in pred]
            scaler = dataset.column_scalers[dataset.target_labels[y_idx]]
            real_sc = dataset.denormalize_values(real,scaler)
            pred_sc = dataset.denormalize_values(pred,scaler)
            res_sc = pred_sc-real_sc
           # Plotting the main data
            plt.subplot(2, 1, 1)  # Create a subplot with 2 rows and 1 column, and select the first subplot
            plt.plot(real_sc, label='Real')  # Add a label for the real data
            plt.plot(pred_sc, label='Predicted')  # Add a label for the predicted data

            # Plotting the residual data
            plt.subplot(2, 1, 2)  # Select the second subplot
            plt.plot(res_sc, label='Residual')  # Add a label for the residual data
            wandb.log({"KWh hourly in 2022": plt})

                        # Display the plot
            plt.show()

                    
    wandb.finish()

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
wandb: Currently logged in as: haradai (energy_project_uab). Use `wandb login --relogin` to force relogin


Epoch 1/200: 71it [00:10,  6.81it/s, Batch Loss=0.00857] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 55.76it/s]


lr:[0.001]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_0.pt


Epoch 2/200: 71it [00:09,  7.30it/s, Batch Loss=0.00318]
Validation: 100%|██████████| 18/18 [00:00<00:00, 88.36it/s]


lr:[0.001]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_0.pt


...

Epoch 199/200: 71it [00:09,  7.34it/s, Batch Loss=2.3e-5]  
Validation: 100%|██████████| 18/18 [00:00<00:00, 81.78it/s]


lr:[1.6423203268260675e-05]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_0.pt


Epoch 200/200: 71it [00:09,  7.34it/s, Batch Loss=2.35e-5] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 89.95it/s]


lr:[1.4780882941434607e-05]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_0.pt
Running whole dataset prediction


Train_loss,█▆▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val_loss,▄█▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_loss,0.00244
Val_loss,2e-05


# Y=1

In [8]:
y_idx = 1
model = attentiveLSTM_model(espai_emb_dim=50,hidden_dim=384,lstm_nl=1,nheads=2,attnFCdim=80)
model.init_weights()
model.to(device)

criterion =  nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #lr used in that 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=(1 - 0.0001 / 0.001))
num_epochs = 200
best_loss = float('inf')
epoch_loss_tr = []
epoch_loss_vl = []
with wandb.init(project="energy_project_uab", entity = "energy_project_uab") as run:
    run.name = f"LSTM_attention_regressor_1"
    for epoch in range(num_epochs):
        model.train()
        training_losses = [] # renamed from epoch_losses
        progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}")
        for batch,data in progress_bar:
            ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
            y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

            optimizer.zero_grad()
            
            #current batch size size
            b_sz = ocu_emb.shape[0]

            #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)
            y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            y_pred = y_pred[:,0,0]

            loss = criterion(y_pred,y)  #cross entropy loss needs (N,C,seq_lenght)
            loss.backward()
            optimizer.step()    

            training_losses.append(loss.item())
            progress_bar.set_postfix({'Batch Loss': loss.item()})

        average_training_loss = sum(training_losses) / len(training_losses) # renamed from avg_loss
        epoch_loss_tr.append(average_training_loss)
        wandb.log({"Train_loss":average_training_loss})
        #average_training_loss = np.power(dataset.denormalize_values(np.sqrt(average_training_loss),scaler),2)

        model.eval()  
        with torch.no_grad():  
            validation_losses = [] # renamed from val_losses
            for batch in tqdm(val_dataloader, desc='Validation'):
                ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
                y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

                #current batch size size
                b_sz = ocu_emb.shape[0]

                #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
                h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
                h = h.to(device)
                c = c.to(device)
                y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
                y_pred = y_pred[:,0,0]
                
                loss = criterion(y_pred,y) 
                validation_losses.append(loss.item())

            average_validation_loss = sum(validation_losses) / len(validation_losses) # renamed from avg_val_loss
            #average_validation_loss = np.power(dataset.denormalize_values(np.sqrt(average_validation_loss),scaler),2)
            epoch_loss_vl.append(average_validation_loss)
            wandb.log({"Val_loss":average_validation_loss})
        
        scheduler.step() #change lr
        print(f"lr:{scheduler.get_last_lr()}")#print current lr

        if average_validation_loss < best_loss:
            best_loss = average_training_loss
            torch.save(model.state_dict(), 'models/attention_exp_LSTM/LSTM_attention_regressor_1.pt')
            wandb.save('models/attention_exp_LSTM/LSTM_attention_regressor_1.pt')
            print("Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_1.pt")
        
        if epoch == num_epochs-1: #if last epoch just finished
            print("Running whole dataset prediction")
            real, pred = predict_consumption(model,whole_dataloader,y_idx)
            real = [v.item() for v in real]
            pred = [v.item() for v in pred]
            scaler = dataset.column_scalers[dataset.target_labels[y_idx]]
            real_sc = dataset.denormalize_values(real,scaler)
            pred_sc = dataset.denormalize_values(pred,scaler)
            res_sc = pred_sc-real_sc
           # Plotting the main data
            plt.subplot(2, 1, 1)  # Create a subplot with 2 rows and 1 column, and select the first subplot
            plt.plot(real_sc, label='Real')  # Add a label for the real data
            plt.plot(pred_sc, label='Predicted')  # Add a label for the predicted data

            # Plotting the residual data
            plt.subplot(2, 1, 2)  # Select the second subplot
            plt.plot(res_sc, label='Residual')  # Add a label for the residual data
            wandb.log({"KWh hourly in 2022": plt})

                        # Display the plot
            plt.show()

                    
    wandb.finish()

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



Epoch 1/200: 71it [00:17,  3.95it/s, Batch Loss=0.016]   
Validation: 100%|██████████| 18/18 [00:00<00:00, 33.33it/s]


lr:[0.001]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_1.pt


Epoch 2/200: 71it [00:12,  5.91it/s, Batch Loss=0.00699] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 46.92it/s]


lr:[0.001]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_1.pt


...

Epoch 86/200: 71it [00:12,  5.76it/s, Batch Loss=0.00143] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 33.01it/s]


lr:[0.00016677181699666576]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_1.pt


Epoch 87/200: 71it [00:13,  5.21it/s, Batch Loss=0.00171] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 35.75it/s]


lr:[0.00016677181699666576]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_1.pt


Epoch 88/200: 56it [00:15,  2.84it/s, Batch Loss=0.000437]

# Y=2

In [ ]:
y_idx = 2
model = attentiveLSTM_model(espai_emb_dim=50,hidden_dim=384,lstm_nl=1,nheads=2,attnFCdim=80)
model.init_weights()
model.to(device)

criterion =  nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #lr used in that 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=(1 - 0.0001 / 0.001))
num_epochs = 200
best_loss = float('inf')
epoch_loss_tr = []
epoch_loss_vl = []
with wandb.init(project="energy_project_uab", entity = "energy_project_uab") as run:
    run.name = f"LSTM_attention_regressor_2"
    for epoch in range(num_epochs):
        model.train()
        training_losses = [] # renamed from epoch_losses
        progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}")
        for batch,data in progress_bar:
            ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
            y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

            optimizer.zero_grad()
            
            #current batch size size
            b_sz = ocu_emb.shape[0]

            #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)
            y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            y_pred = y_pred[:,0,0]

            loss = criterion(y_pred,y)  #cross entropy loss needs (N,C,seq_lenght)
            loss.backward()
            optimizer.step()    

            training_losses.append(loss.item())
            progress_bar.set_postfix({'Batch Loss': loss.item()})

        average_training_loss = sum(training_losses) / len(training_losses) # renamed from avg_loss
        epoch_loss_tr.append(average_training_loss)
        wandb.log({"Train_loss":average_training_loss})
        #average_training_loss = np.power(dataset.denormalize_values(np.sqrt(average_training_loss),scaler),2)

        model.eval()  
        with torch.no_grad():  
            validation_losses = [] # renamed from val_losses
            for batch in tqdm(val_dataloader, desc='Validation'):
                ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
                y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

                #current batch size size
                b_sz = ocu_emb.shape[0]

                #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
                h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
                h = h.to(device)
                c = c.to(device)
                y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
                y_pred = y_pred[:,0,0]
                
                loss = criterion(y_pred,y) 
                validation_losses.append(loss.item())

            average_validation_loss = sum(validation_losses) / len(validation_losses) # renamed from avg_val_loss
            #average_validation_loss = np.power(dataset.denormalize_values(np.sqrt(average_validation_loss),scaler),2)
            epoch_loss_vl.append(average_validation_loss)
            wandb.log({"Val_loss":average_validation_loss})
        
        scheduler.step() #change lr
        print(f"lr:{scheduler.get_last_lr()}")#print current lr

        if average_validation_loss < best_loss:
            best_loss = average_training_loss
            torch.save(model.state_dict(), 'models/attention_exp_LSTM/LSTM_attention_regressor_2.pt')
            wandb.save('models/attention_exp_LSTM/LSTM_attention_regressor_2.pt')
            print("Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_2.pt")
        
        if epoch == num_epochs-1: #if last epoch just finished
            print("Running whole dataset prediction")
            real, pred = predict_consumption(model,whole_dataloader,y_idx)
            real = [v.item() for v in real]
            pred = [v.item() for v in pred]
            scaler = dataset.column_scalers[dataset.target_labels[y_idx]]
            real_sc = dataset.denormalize_values(real,scaler)
            pred_sc = dataset.denormalize_values(pred,scaler)
            res_sc = pred_sc-real_sc
           # Plotting the main data
            plt.subplot(2, 1, 1)  # Create a subplot with 2 rows and 1 column, and select the first subplot
            plt.plot(real_sc, label='Real')  # Add a label for the real data
            plt.plot(pred_sc, label='Predicted')  # Add a label for the predicted data

            # Plotting the residual data
            plt.subplot(2, 1, 2)  # Select the second subplot
            plt.plot(res_sc, label='Residual')  # Add a label for the residual data
            wandb.log({"KWh hourly in 2022": plt})

                        # Display the plot
            plt.show()

                    
    wandb.finish()

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



Epoch 1/200: 71it [00:17,  4.16it/s, Batch Loss=0.00417] 
Validation: 100%|██████████| 18/18 [00:00<00:00, 38.67it/s]


lr:[0.001]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_2.pt


Epoch 2/200: 71it [00:10,  6.47it/s, Batch Loss=0.000797]
Validation: 100%|██████████| 18/18 [00:00<00:00, 46.73it/s]


lr:[0.001]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_2.pt


...

Epoch 199/200: 71it [00:10,  6.65it/s, Batch Loss=0.000106]
Validation: 100%|██████████| 18/18 [00:00<00:00, 46.00it/s]


lr:[1.6423203268260675e-05]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_2.pt


Epoch 200/200: 71it [00:10,  6.67it/s, Batch Loss=0.000105]
Validation: 100%|██████████| 18/18 [00:00<00:00, 42.81it/s]


lr:[1.4780882941434607e-05]
Model saved at models/attention_exp_LSTM/LSTM_attention_regressor_2.pt
Running whole dataset prediction


Train_loss,▆█▅▅▄█▆█▇▇▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val_loss,▁▁▂▁▁▂▁█▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_loss,0.00201
Val_loss,0.0001


We thought that instead of doing that manually by trying and error we will create a second model, given this first one is frozen that will try to minimize the energy consumption by given some input of activities generating the Espai where it should be hold in. We'll put in an important loss for repeating the same espai for different activities. 

Then we'll do the same but for the opposite, make it attempt to using the same mechanism make the energy consumption as high as possible. This way with both, we'll be able to detect what "espais" are related with more consumption.

We could further do the same for activities but we are going to keep it simple for now...

## Model: